In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
!pip install stable-baselines3[extra]
!pip install gymnasium

In [ ]:
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

In [5]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [6]:
env_id = "PandaReachDense-v3"
env = gym.make(env_id)
s_size = env.observation_space.shape
a_size = env.action_space

In [7]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('achieved_goal', array([-9.997612 , -4.4509263, -9.282204 ], dtype=float32)), ('desired_goal', array([ 4.128272 , -0.4184126,  2.9989922], dtype=float32)), ('observation', array([ 0.9515511 ,  3.6744523 , -3.5884829 ,  3.4498413 ,  9.938854  ,
       -0.90597683], dtype=float32))])


In [8]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [ 0.30665144 -0.92529     0.35791552]


In [9]:
env = make_vec_env(env_id, n_envs=4)

# Adding this wrapper to normalize the observation and the reward
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

In [11]:
model = A2C(policy="MultiInputPolicy", env=env, verbose=1)

Using cuda device


In [12]:
model.learn(1_000_000)

Streaming output truncated to the last 5000 lines.
| time/                 |          |
|    fps                | 344      |
|    iterations         | 22300    |
|    time_elapsed       | 1295     |
|    total_timesteps    | 446000   |
| train/                |          |
|    entropy_loss       | -1.13    |
|    explained_variance | 0.953    |
|    learning_rate      | 0.0007   |
|    n_updates          | 22299    |
|    policy_loss        | 0.000131 |
|    std                | 0.369    |
|    value_loss         | 0.000132 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.69     |
|    ep_rew_mean        | -0.211   |
| time/                 |          |
|    fps                | 344      |
|    iterations         | 22400    |
|    time_elapsed       | 1301     |
|    total_timesteps    | 448000   |
| train/                |          |
|    entropy_loss       | -1.11    |
|    explained_variance 

In [13]:
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

In [14]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward = -0.18 +/- 0.12


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [15]:
notebook_login()
!git config --global credential.helper store

In [16]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"kowalsky/a2c-{env_id}",
    commit_message="Initial commit",
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmpzwv_s_to/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpzwv_s_to/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpzwv_s_to/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpzwv_s_to/-step-0-to-step-1000.mp4
ℹ Pushing repo kowalsky/a2c-PandaReachDense-v3 to the Hugging Face
Hub


...

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/kowalsky/a2c-PandaReachDense-v3/tree/main/


'https://huggingface.co/kowalsky/a2c-PandaReachDense-v3/tree/main/'